In [7]:
pip install -U openai-whisper

In [8]:
!pip install pytube

In [9]:
!pip install nltk

In [10]:
import os
import whisper
from pytube import YouTube
import datetime
import tempfile
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hoang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hoang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hoang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hoang\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:

# Tải mô hình Whisper
model = whisper.load_model('base')

def transcribe_video_to_text(file_path):
    # Kiểm tra sự tồn tại của tệp
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"The video {file_path} does not exist.")

    # Chuyển đổi âm thanh thành văn bản
    result = model.transcribe(file_path)

    # Tạo một chuỗi để lưu kết quả
    transcription = ""
    for indx, segment in enumerate(result['segments']):
        start_time = str(datetime.timedelta(seconds=int(segment['start'])))
        end_time = str(datetime.timedelta(seconds=int(segment['end'])))
        transcription += str(indx + 1) + '\n'
        transcription += start_time + ' --> ' + end_time + '\n'
        transcription += segment['text'] + '\n\n'

    return transcription

def download_youtube_audio_to_temp(youtube_link):
    yt = YouTube(youtube_link)
    stream = yt.streams.filter(only_audio=True).first()
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp4')
    stream.download(filename=temp_file.name)
    return temp_file.name

def get_wordnet_pos(word_tag):
    """Map POS tag to first character lemmatize() accepts"""
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV
    }
    return tag_dict.get(word_tag[0].upper(), wordnet.NOUN)

def preprocess_text(text):
    # Chuyển đổi văn bản thành chữ thường
    text = text.lower()

    # Loại bỏ các từ viết tắt thường gặp
    abbreviations = {
        r"(\w+)'s": r"\1",
        r"(\w+)'m": r"\1",
        r"(\w+)'ve": r"\1",
        r"(\w+)'t": r"\1",
        r"(\w+)'re": r"\1",
        r"(\w+)'d": r"\1",
        r"(\w+)'ll": r"\1",
        r"n't": "",
    }

    for pattern, repl in abbreviations.items():
        text = re.sub(pattern, repl, text)

    # Tách từ và phân tích POS
    words = word_tokenize(text)
    pos_tags = nltk.pos_tag(words)

    # Loại bỏ stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word, tag in pos_tags if word not in stop_words]

    # Chuẩn hóa từ về dạng gốc
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags if word not in stop_words]

    # Kết hợp các từ thành văn bản mới
    processed_text = ' '.join(words)

    return processed_text

# Tiền xử lý văn bản sau khi đã chuyển từ âm thanh
def preprocess_transcribed_text(transcribed_text):
    processed_text = preprocess_text(transcribed_text)
    return processed_text

def preprocess_and_save_transcription(transcription):
    processed_text = preprocess_transcribed_text(transcription)

    with open('processed_text.txt', 'w') as file:
        file.write(processed_text)

# Kết quả sau khi xử lý sẽ được lưu vào đây
processed_transcription = ""

# Sử dụng video cục bộ
video_path = 'video.mp4'
try:
    local_transcription = transcribe_video_to_text(video_path)
    processed_local_transcription = preprocess_transcribed_text(local_transcription)
    processed_transcription += "Local video transcription:\n"
    processed_transcription += processed_local_transcription
except FileNotFoundError as e:
    processed_transcription += str(e) + "\n"

# Sử dụng link YouTube
youtube_link = 'https://www.youtube.com/watch?v=LHVTHGtNeic'
try:
    temp_audio_file = download_youtube_audio_to_temp(youtube_link)
    youtube_transcription = transcribe_video_to_text(temp_audio_file)
    processed_youtube_transcription = preprocess_transcribed_text(youtube_transcription)
    processed_transcription += "\nYouTube video transcription:\n"
    processed_transcription += processed_youtube_transcription
    os.remove(temp_audio_file)  # Xóa tệp tạm thời sau khi hoàn tất
except FileNotFoundError as e:
    processed_transcription += str(e) + "\n"

# In ra kết quả
print(processed_transcription)



C:\Users\hoang\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


The video video.mp4 does not exist.

YouTube video transcription:
1 0:00:00 -- > 0:00:04 fuck way . 2 0:00:04 -- > 0:00:07 fuck ? 3 0:00:07 -- > 0:00:12 donate , five year ago , send mug , mug old 4 0:00:12 -- > 0:00:16 pee wear , say b , fucking apt . 5 0:00:16 -- > 0:00:19 yeah , always either prepare colonoscopy come back 6 0:00:19 -- > 0:00:20 colonoscopy . 7 0:00:20 -- > 0:00:22 fuck go ? 8 0:00:22 -- > 0:00:23 woodstock overassholes ! 9 0:00:23 -- > 0:00:26 damn , hate new stair ! 10 0:00:26 -- > 0:00:30 oh , , , fuck ! 11 0:00:30 -- > 0:00:31 fuck , get fuck ! 12 0:00:31 -- > 0:00:32 fuck , fuck ! 13 0:00:32 -- > 0:00:36 oh , god , disgusting , think gon na puke ! 14 0:00:36 -- > 0:00:37 fuck ! 15 0:00:37 -- > 0:00:42 peter , say volunteer shit ? 16 0:00:42 -- > 0:00:44 pretty sure heard people call spidey . 17 0:00:44 -- > 0:00:45 yeah , close friend . 18 0:00:45 -- > 0:00:46 name ? 19 0:00:46 -- > 0:00:47 peter ? 20 0:00:47 -- > 0:00:48 hey , pety , stuff . 21 0:00:48 -- > 0:0

In [12]:
#In văn bản đã được tiền xử lí
print(processed_transcription)

The video video.mp4 does not exist.

YouTube video transcription:
1 0:00:00 -- > 0:00:04 fuck way . 2 0:00:04 -- > 0:00:07 fuck ? 3 0:00:07 -- > 0:00:12 donate , five year ago , send mug , mug old 4 0:00:12 -- > 0:00:16 pee wear , say b , fucking apt . 5 0:00:16 -- > 0:00:19 yeah , always either prepare colonoscopy come back 6 0:00:19 -- > 0:00:20 colonoscopy . 7 0:00:20 -- > 0:00:22 fuck go ? 8 0:00:22 -- > 0:00:23 woodstock overassholes ! 9 0:00:23 -- > 0:00:26 damn , hate new stair ! 10 0:00:26 -- > 0:00:30 oh , , , fuck ! 11 0:00:30 -- > 0:00:31 fuck , get fuck ! 12 0:00:31 -- > 0:00:32 fuck , fuck ! 13 0:00:32 -- > 0:00:36 oh , god , disgusting , think gon na puke ! 14 0:00:36 -- > 0:00:37 fuck ! 15 0:00:37 -- > 0:00:42 peter , say volunteer shit ? 16 0:00:42 -- > 0:00:44 pretty sure heard people call spidey . 17 0:00:44 -- > 0:00:45 yeah , close friend . 18 0:00:45 -- > 0:00:46 name ? 19 0:00:46 -- > 0:00:47 peter ? 20 0:00:47 -- > 0:00:48 hey , pety , stuff . 21 0:00:48 -- > 0:0

In [13]:
import re

# Hàm tách đoạn văn thành các dòng dựa trên số thứ tự
def split_lines_by_numbered_prefix(text):
    lines = text.strip().split()
    result = []
    current_line = []
    current_number = 1

    for part in lines:
        if part.isdigit() and int(part) == current_number:
            if current_line:
                result.append(' '.join(current_line))
                current_line = []
            current_line.append(part)
            current_number += 1
        else:
            current_line.append(part)

    if current_line:
        result.append(' '.join(current_line))

    return result


# Chia đoạn văn thành các dòng dựa trên số thứ tự
split_lines = split_lines_by_numbered_prefix(processed_transcription)

# Ghép các dòng lại thành chuỗi
split_text = '\n'.join(split_lines)

# In nội dung đã tách dòng
print(split_text)



The video video.mp4 does not exist. YouTube video transcription:
1 0:00:00 -- > 0:00:04 fuck way .
2 0:00:04 -- > 0:00:07 fuck ?
3 0:00:07 -- > 0:00:12 donate , five year ago , send mug , mug old
4 0:00:12 -- > 0:00:16 pee wear , say b , fucking apt .
5 0:00:16 -- > 0:00:19 yeah , always either prepare colonoscopy come back
6 0:00:19 -- > 0:00:20 colonoscopy .
7 0:00:20 -- > 0:00:22 fuck go ?
8 0:00:22 -- > 0:00:23 woodstock overassholes !
9 0:00:23 -- > 0:00:26 damn , hate new stair !
10 0:00:26 -- > 0:00:30 oh , , , fuck !
11 0:00:30 -- > 0:00:31 fuck , get fuck !
12 0:00:31 -- > 0:00:32 fuck , fuck !
13 0:00:32 -- > 0:00:36 oh , god , disgusting , think gon na puke !
14 0:00:36 -- > 0:00:37 fuck !
15 0:00:37 -- > 0:00:42 peter , say volunteer shit ?
16 0:00:42 -- > 0:00:44 pretty sure heard people call spidey .
17 0:00:44 -- > 0:00:45 yeah , close friend .
18 0:00:45 -- > 0:00:46 name ?
19 0:00:46 -- > 0:00:47 peter ?
20 0:00:47 -- > 0:00:48 hey , pety , stuff .
21 0:00:48 -- > 0:00

In [17]:
keyword_dict = {
"nội dung người lớn và các thuật ngữ tình dục": ["#freethenipple", "18+", "ampland", "anal", "anus", "arse", "ass", "asslick", "ball licking", "ball sucking", "bangbros", "bangbus", "banged", "barely legal", "bastinado", "bdsm", "bestiality", "bigtits", "bitch", "blow job", "blowjob", "blowjobs", "bondage", "boner", "boob", "boobies", "booboooooobs", "boobs", "bosomy", "bra", "brazzers", "buceta", "bukkake", "buttfuck", "butthole", "buttlick", "buttsex", "cam girl", "cameltoe", "censored", "clit", "clit licker", "clitoris", "clits", "cock", "cockhead", "cocksucker", "cocksuckers", "coom", "coomer", "cum", "cumguzzler", "cumming", "cumshot", "cumshots", "cumslut", "cunnilingus", "dildo", "dildos", "doggie style", "doggiestyle", "doggy style", "dominatrix", "dommes", "dry hump", "ecchi", "erotic", "erotica", "fetish", "fetishes", "fingered", "fingerfuck", "fingerfucked", "fingerfucks", "fingering", "fistfuck", "fistfucked", "fistfuckers", "fondle", "fondled", "footjob", "genital", "genitals", "girl on top", "gratis", "gratuit", "grope", "groped", "groping", "hand job", "handjob", "handjobs", "hardcoresex", "hem", "hentai", "horney", "horny", "hot", "hot carl", "hot sex", "hotbox", "hotsex", "hottest", "huge", "humping", "incest", "incl", "intercourse", "interracial", "invisible", "kalergi", "labia", "latex", "lesbians", "lezzie", "licking", "lolicon", "lolita", "masturbate", "masturbating", "masturbation", "masturbators", "milf", "milf hunter", "milfs", "nipple", "nipples", "nude", "nudes", "nudist", "nudity", "oral sex", "orgasim", "orgasm", "orgasms", "orgies", "orgy", "panties", "panty", "pantyhose", "penis", "pervert", "perverted", "perverts", "phonesex", "pingas", "porn", "pornhub", "porno", "pornography", "pornos", "prostitution", "pussies", "pussy", "pussylicking", "r18", "redhead", "reverse cowgirl", "rimjob", "rimming", "seksi", "semen", "sex", "sex work", "sex worker", "sexo", "sexual health", "sexually", "sexy", "sexy time", "shibari", "shoplifter", "slut", "sluts", "spycam", "squirt", "squirting", "strapon", "strip", "strip club", "stripper", "stripping", "succubus", "suck", "sucking", "teens", "thong", "threesome", "throating", "titfuck", "tities", "tits", "titten", "tittie", "titties", "titty", "topless", "tribadism", "twat", "twats", "undress", "undressing", "upskirt", "upskirts", "vagina", "vaginal", "vaginal thrush", "viagra", "voyeur", "vulva", "watch free", "watch online", "webcam", "x-rated", "xxx", "xxx-tentacion"],
"thuyết âm mưu": ["pizzagate", "qanon"],
"tội phạm và bạo lực": ["9-11", "9/11", "abuser", "abusing", "abusive", "accused", "adolescent", "allegation", "allegations", "assaulted", "boko haram", "bullying", "chester bennington", "columbine", "convicted", "crimes", "cruel", "dead body", "deaths", "el chapo", "elliot rodger", "epstein", "epstein didn't kill himself", "execution", "isil", "isis", "iyad el-baghdadi", "kidnap", "murder", "murdered", "punishments", "rape", "raped", "raping", "rapist", "robbery", "sandy hook", "slaughtering", "suspect", "ted bundy", "terrorism", "terrorist", "terrorists", "violence", "violently", "virginia tech shooting", "harass", "harassment", "hazing", "betting", "child pornography", "counterfeit", "gambling", "hacking", "pedophile", "snuff"],
"ma túy và chất kích thích": ["cannabis", "cannabutter", "cocaine", "crack", "crackwhore", "drug", "drugs", "ecstasy", "hemp", "lsd", "marijuana", "meth", "pcp", "salvia", "stoned", "stoner", "thc", "vape", "weed"],
"các tổ chức cực đoan":["ku klux klan", "nazi", "neonazi"],
"dịch bệnh và bệnh tật":["coronavirus", "covid-19", "ebola", "aids", "bladder cancer", "breast cancer", "cancer", "cyst", "erectile dysfunction", "escherichia coli", "gallbladder cancer", "hiv", "lung cancer", "thyroid cancer"],
"các vấn đề chính trị và xã hội": ["#metoo", "8chan", "abortion", "abortions", "al qaeda", "al qaida", "antifa", "blue anon", "democrat", "democratic", "democrats", "election", "election fraud", "elections", "electoral", "hunter biden", "pizzagate", "pro-choice", "qanon", "voter fraud"],
"lời lẽ tục tĩu và xúc phạm": ["asshole", "assholes", "bitches", "blyat", "chuj", "cunt", "dick", "dickhead", "fack", "fag", "fagged", "faggit", "faghag", "fags", "fuck", "fuckboy", "fucked", "fucker", "fuckers", "fuckin", "fuckings", "fucks", "fucky", "fuk", "gayass", "gayfuck", "gaysex", "idiot", "im rick james bitch", "kurwa", "mothafucka", "mothafuckas", "mothafucker", "mothafuckers", "mothafucking", "motherfucka", "motherfucker", "motherfuckers", "motherfuckin", "motherfucking", "n1gga", "n1gger", "nasty", "nigga", "niggah", "niggas", "niggaz", "nip", "puta", "shit", "shite", "shitfuck", "shitpost", "shits", "shitted", "shitting", "shitty", "sonofabitch", "stfu", "whore", "whores", "yvonne"],
"phân biệt chủng tộc và kỳ thị": ["chingada madre", "chink", "jews", "jigaboo", "jiggaboo", "kike", "nigger", "porch monkey", "porchmonkey", "racist", "white pride", "white supremacists"],
"hành vi nguy hiểm và tự hại": ["anthony bourdain", "bulimia", "cut", "cutting", "eugenia cooney", "momo", "suicide"]
}

In [18]:
print (split_text)


The video video.mp4 does not exist. YouTube video transcription:
1 0:00:00 -- > 0:00:04 fuck way .
2 0:00:04 -- > 0:00:07 fuck ?
3 0:00:07 -- > 0:00:12 donate , five year ago , send mug , mug old
4 0:00:12 -- > 0:00:16 pee wear , say b , fucking apt .
5 0:00:16 -- > 0:00:19 yeah , always either prepare colonoscopy come back
6 0:00:19 -- > 0:00:20 colonoscopy .
7 0:00:20 -- > 0:00:22 fuck go ?
8 0:00:22 -- > 0:00:23 woodstock overassholes !
9 0:00:23 -- > 0:00:26 damn , hate new stair !
10 0:00:26 -- > 0:00:30 oh , , , fuck !
11 0:00:30 -- > 0:00:31 fuck , get fuck !
12 0:00:31 -- > 0:00:32 fuck , fuck !
13 0:00:32 -- > 0:00:36 oh , god , disgusting , think gon na puke !
14 0:00:36 -- > 0:00:37 fuck !
15 0:00:37 -- > 0:00:42 peter , say volunteer shit ?
16 0:00:42 -- > 0:00:44 pretty sure heard people call spidey .
17 0:00:44 -- > 0:00:45 yeah , close friend .
18 0:00:45 -- > 0:00:46 name ?
19 0:00:46 -- > 0:00:47 peter ?
20 0:00:47 -- > 0:00:48 hey , pety , stuff .
21 0:00:48 -- > 0:00

In [19]:
import re

# Hàm phân loại văn bản và tìm vị trí các từ vi phạm
def classify_text_with_times(text, keyword_dict):
    labels = set()  # Sử dụng set để tránh nhãn trùng lặp
    time_positions = {}  # Từ điển để lưu trữ thời gian của các từ vi phạm

    # Tách đoạn văn bản theo từng dòng
    lines = text.strip().split("\n")

    for line in lines:
        if '-- >' in line:
            parts = line.split("-- >")
            time_range = parts[0].strip().split()[1] + " -- > " + parts[1].strip().split()[0]
            transcript = " ".join(parts[1].strip().split()[1:])
            cleaned_transcript = re.sub(r'[^\w\s]', '', transcript.lower())

            for label, phrases in keyword_dict.items():
                for phrase in phrases:
                    escaped_phrase = re.escape(phrase)
                    if re.search(r'\b' + escaped_phrase + r'\b', cleaned_transcript):
                        labels.add(label)
                        if label in time_positions:
                            time_positions[label].append(time_range)
                        else:
                            time_positions[label] = [time_range]
                        print(f"Cụm từ '{phrase}' khớp với nhãn '{label}' tại thời gian {time_range}")

    return list(labels), time_positions
# Phân loại đoạn văn bản và lấy thời gian các từ vi phạm
labels, time_positions = classify_text_with_times(split_text, keyword_dict)
print("Nhãn vi phạm:", labels)
print("Thời gian vi phạm:", time_positions)



Cụm từ 'fuck' khớp với nhãn 'lời lẽ tục tĩu và xúc phạm' tại thời gian 0:00:00 -- > 0:00:04
Cụm từ 'fuck' khớp với nhãn 'lời lẽ tục tĩu và xúc phạm' tại thời gian 0:00:04 -- > 0:00:07
Cụm từ 'fuck' khớp với nhãn 'lời lẽ tục tĩu và xúc phạm' tại thời gian 0:00:20 -- > 0:00:22
Cụm từ 'fuck' khớp với nhãn 'lời lẽ tục tĩu và xúc phạm' tại thời gian 0:00:26 -- > 0:00:30
Cụm từ 'fuck' khớp với nhãn 'lời lẽ tục tĩu và xúc phạm' tại thời gian 0:00:30 -- > 0:00:31
Cụm từ 'fuck' khớp với nhãn 'lời lẽ tục tĩu và xúc phạm' tại thời gian 0:00:31 -- > 0:00:32
Cụm từ 'fuck' khớp với nhãn 'lời lẽ tục tĩu và xúc phạm' tại thời gian 0:00:36 -- > 0:00:37
Cụm từ 'shit' khớp với nhãn 'lời lẽ tục tĩu và xúc phạm' tại thời gian 0:00:37 -- > 0:00:42
Cụm từ 'fuck' khớp với nhãn 'lời lẽ tục tĩu và xúc phạm' tại thời gian 0:00:50 -- > 0:00:54
Cụm từ 'idiot' khớp với nhãn 'lời lẽ tục tĩu và xúc phạm' tại thời gian 0:00:56 -- > 0:00:58
Cụm từ 'fuck' khớp với nhãn 'lời lẽ tục tĩu và xúc phạm' tại thời gian 0:01:00 